# Epidemic Model 

        In this lab, we use optimal control techniques to find a vaccination schedule for an epidemic disease. 
        Our goal is to minimize the number of infectious persons and the overall cost of the vaccine during a fixed time period.

### Modelo Escolhido (SEIR)

        Our model considers three compartiments that represent three groups over the population, each one is considered homogeneous. Let $S(t), R(t)$ and $I(t)$ be the groups representing susceptible, recovered (immune) and infectious (note the difference from infecteds).
        The SEIR Model allows a incubation period foir the disease, and the infected person remains latent for some time before becoming infectious. Let $E(t)$ be this number. $N(t) = S(t) + R(t) + I(t) + E(t)$. 
        Let $u(t)$ be the control that represents the percentage of susceptible individuals being vaccinated per unit of time. $0 \leq u(t) \leq 0.9$. $b$ is the natural exponential birth rate and $d$ natural death. The incidence pf the disease is $cS(t)I(t)$ and $e$ is the tate at which the exposed infividual become infectious. And $g$ the recover rate. 

$$ min_u \int_0^T AI(t) + u(t)^2 dt \iff max_u \int_0^T - AI(t) - u(t)^2 dt $$ 

$$ s.a ~~ S'(t) = bN(t) - dS(t) - cS(t)I(t) - u(t)S(t), S(0) = S_0 \geq 0 $$

$$ E'(t) = cS(t)I(t) - (e + d)E(t), E(0) = E_0 \geq 0 $$

$$I'(t) = eE(t) - (g + a + d)I(t), I(0) = I_0 \geq 0$$

$$R'(t) = gI(t) - dR(t) + u(t)S(t), R(0) = R_0 \geq 0$$

$$N'(t) = (b - d)N(t) - aI(t), N(0) = N_0$$

$$0\leq u(t) \leq 0.9$$

## Functions

In [2]:
f = lambda A,I,u: - A*I - u**2
g_S = lambda b,c,d,S,E,I,R,u: b*(S + E + I + R) - d*S - c*S*I - u*S
g_E = lambda c,e,d,S,E,I,R,u: c*S*I - (e + d)*E
g_I = lambda a,d,e,g,S,E,I,R,u: e*E - (g + a + d)*I
g_R = lambda d,g,S,E,I,R,u: g*I - d*R + u*S 

Sejam $\lambda_1(t),\lambda_2(t),\lambda_3(t),\lambda_4(t)$ funções deriváveis por partes
   
$\frac{\partial H}{\partial S} = \lambda_1(b - d - cI - u) + \lambda_2(cI) + \lambda_4(u) = - \lambda_1'(t)$

$\frac{\partial H}{\partial E} = \lambda_1(b) - \lambda_2(e + d) + \lambda_3(e) = - \lambda_2'(t)$

$\frac{\partial H}{\partial I} = - A + \lambda_1(b - cS) + \lambda_2(cS) - \lambda_3(g + a + d) + \lambda_4(g) = - \lambda_3'(t)$

$\frac{\partial H}{\partial R} = \lambda_1(b) - \lambda_4(d) = - \lambda_4'(t)$

Note que $\lambda_1(1) = \lambda_2(1) = \lambda_3(1) = \lambda_4(1) = 0 $

$\frac{\partial H}{\partial u} = -2u - \lambda_1(S) + \lambda_4(S)$

$\frac{\partial H}{\partial u} < 0 \Rightarrow u(t) = 0 \Rightarrow -2u - \lambda_1(S) + \lambda_4(S) < 0 \Rightarrow S(\lambda_4 - \lambda_1) < 0 \Rightarrow \frac{1}{2}S(\lambda_4 - \lambda_1) < 0 $

$\frac{\partial H}{\partial u} > 0 \Rightarrow u(t) = 0.9 \Rightarrow -2u - \lambda_1(S) + \lambda_4(S) > 0 \Rightarrow S(\lambda_4 - \lambda_1) > 1.8 \Rightarrow \frac{1}{2}S(\lambda_4 - \lambda_1) > 0.9$

$\frac{\partial H}{\partial u} = 0 \Rightarrow 0 \leq u(t) \leq 0.9 \Rightarrow -2u - \lambda_1(S) + \lambda_4(S) = 0 \Rightarrow u(t) = \frac{1}{2}S(\lambda_4 - \lambda_1) \Rightarrow 0 \leq \frac{1}{2}S(\lambda_4 - \lambda_1) \leq 0.9$

Portanto $u^*(t) = \max(0, \min(\frac{1}{2}S(\lambda_4 - \lambda_1),0.9)) $

In [3]:
class solve:
    
    def __init__(self,A,a,b,c,d,e,g,T,S0,E0,I0,R0):
        
        self.A = A
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.e = e
        self.g = g
        self.T = T
        self.S0 = S0
        self.E0 = E0
        self.I0 = I0
        self.R0 = R0
        
    Fu = lambda S,E,I,R,u,adj: max(0,min(1/2*S*(adj[3] - adj[0]),0.9))
        
    g_S = lambda S,E,I,R,u: self.b*(S + E + I + R) - self.d*S - self.c*S*I - u*S
    g_E = lambda S,E,I,R,u: self.c*S*I - (self.e + self.d)*E
    g_I = lambda S,E,I,R,u: self.e*E - (self.g + self.a + self.d)*I
    g_R = lambda S,E,I,R,u: self.g*I - self.d*R + u*S
    
    dadj1 = lambda S,E,I,R,u,adj: -adj[0]*(self.b - self.d - self.c*I - u) + -adj[1]*self.c*I - adj[3]*u
    dadj2 = lambda S,E,I,R,u,adj: -adj[0]*self.b + adj[1]*(self.e + self.d) - adj[2]*self.e
    dadj3 = lambda S,E,I,R,u,adj: self.A - adj[0]*(self.b - self.c*S) - adj[1]*self.c*S + 
                                  adj[2]*(self.g + self.a + self.d) - adj[3]*g 
    dadj4 = lambda S,E,I,R,u,adj: -adj[0]*b + adj[3]*d 
    
    def runge_kutta_state(self,t,S,E,I,R,adj,N,h):

        for i in range(N):
            
            k1 = self.dx(t[k],x[k],adj[k])
            k2 = self.dx(t[k]+h/2,x[k] + h*k1/2,adj[k])
            k3 = self.dx(t[k]+h/2,x[k]+h*k2/2,adj[k])
            k4 = self.dx(t[k] + h, x[k] + h*k3, adj[k])
            x[k+1] = x[k] + (h/6)*(k1 + 2*k2 + 2*k3 + k4)
        return x

    def runge_kutta_adj(self,t,S,E,I,R,adj,N,h):
    
        for k in range(N,0,-1):
            k1 = self.dadj(t[k],x[k],adj[k])
            k2 = self.dadj(t[k] - h/2,x[k],adj[k] + h*k1/2)
            k3 = self.dadj(t[k] - h/2,x[k],adj[k]+h*k2/2)
            k4 = self.dadj(t[k] - h, x[k],adj[k] + h*k3)
            adj[k-1] = adj[k] - (h/6)*(k1 + 2*k2 + 2*k3 + k4)
        return adj